In [31]:
# Run this code in conda environment "MDN" to make MDN part working correctly.

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [33]:
#pd.set_option('display.max_rows' , 500)

#  field chla data preparation

In [34]:
field_data = pd.read_csv('../Data/BP_data.csv', skiprows=[1], parse_dates=['Date'], index_col= 'SampleID')
field_data

,Dataset,Date,Time,SiteNumber,latitude,Longitude,BottomDepth,sampledepth,SecchiDiskDepth,SamplingInScum,...,Diatom_Synedra,Diatom_Cocconeis,Diatom_Nitzchia,Diatom_Navicula,Diatom_Fragillaria,Diatom_Tabellaria,Diatom_Gomphenema,Diatom_Astrionella,Flagellates,Other
SampleID,,,,,,,,,,,,,,,,,,,,,
1,Buoy,2014-06-11,6:15:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Buoy,2014-06-11,6:30:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Buoy,2014-06-11,6:45:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Buoy,2014-06-11,7:00:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Buoy,2014-06-11,7:15:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110085,Calibration,2019-08-30,11:37:00 AM,1,50.586139,-105.384833,4.3,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110086,Calibration,2019-09-03,12:05:00 PM,1,50.586139,-105.384833,4.3,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110087,Calibration,2019-09-03,12:05:00 PM,1,50.586139,-105.384833,4.3,0.8,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#buoy calibration
filt_dataset = field_data['Dataset'] == 'Buoy'
field_data.loc[filt_dataset, 'Lab_Chla'] = (5 * field_data[filt_dataset]['BGAPCShallowRFU']) ** (100/99)
field_data['Lab_Chla'].head()

SampleID
1    18.533269
2    17.493540
3    16.454429
4    19.053354
5    16.454429
Name: Lab_Chla, dtype: float64

In [36]:
# drop the buoy observations associated with biofouling error
field_data = field_data[field_data['shallow_sonde_biofouling_outlier'] != 1].copy()
# drop the associated column 
field_data.drop('shallow_sonde_biofouling_outlier', axis=1, inplace=True)

In [37]:
# drop the lab observations labeld as error
field_data = field_data[field_data.Lab_chlaOutlier != 1].copy()
# drop the associated column 
field_data.drop('Lab_chlaOutlier', axis=1, inplace=True)
field_data

,Dataset,Date,Time,SiteNumber,latitude,Longitude,BottomDepth,sampledepth,SecchiDiskDepth,SamplingInScum,...,Diatom_Synedra,Diatom_Cocconeis,Diatom_Nitzchia,Diatom_Navicula,Diatom_Fragillaria,Diatom_Tabellaria,Diatom_Gomphenema,Diatom_Astrionella,Flagellates,Other
SampleID,,,,,,,,,,,,,,,,,,,,,
1,Buoy,2014-06-11,6:15:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Buoy,2014-06-11,6:30:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Buoy,2014-06-11,6:45:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Buoy,2014-06-11,7:00:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Buoy,2014-06-11,7:15:00 PM,1,50.586139,-105.384833,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110085,Calibration,2019-08-30,11:37:00 AM,1,50.586139,-105.384833,4.3,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110086,Calibration,2019-09-03,12:05:00 PM,1,50.586139,-105.384833,4.3,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110087,Calibration,2019-09-03,12:05:00 PM,1,50.586139,-105.384833,4.3,0.8,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# averaging field chla data over date and station
chl_data = field_data.groupby(['Date', 'SiteNumber', 'latitude', 'Longitude']).agg({'Lab_Chla' :'median'})
chl_data

,,,,Lab_Chla
Date,SiteNumber,latitude,Longitude,
2014-01-06,2,50.585203,-105.385823,NaN
2014-01-13,2,50.585203,-105.385823,6.000000
2014-01-20,2,50.585203,-105.385823,NaN
2014-01-27,2,50.585203,-105.385823,7.000000
2014-02-03,2,50.585203,-105.385823,NaN
...,...,...,...,...
2019-09-29,1,50.586139,-105.384833,38.898296
2019-09-30,1,50.586139,-105.384833,39.946201
2019-10-01,1,50.586139,-105.384833,39.946201


In [39]:
# drop (dates,stations) with no chla measured
chl_data.dropna(axis=0, inplace=True)
chl_data

,,,,Lab_Chla
Date,SiteNumber,latitude,Longitude,
2014-01-13,2,50.585203,-105.385823,6.000000
2014-01-27,2,50.585203,-105.385823,7.000000
2014-02-10,2,50.585203,-105.385823,19.000000
2014-02-24,2,50.585203,-105.385823,14.000000
2014-03-10,2,50.585203,-105.385823,8.000000
...,...,...,...,...
2019-09-29,1,50.586139,-105.384833,38.898296
2019-09-30,1,50.586139,-105.384833,39.946201
2019-10-01,1,50.586139,-105.384833,39.946201


In [40]:
chl_data.reset_index(level=['Date', 'SiteNumber', 'latitude', 'Longitude'], inplace=True)
chl_data.index
chl_data.dtypes

Date          datetime64[ns]
SiteNumber             int64
latitude             float64
Longitude            float64
Lab_Chla             float64
dtype: object

# Extract satelite data

In [41]:
# create a dataframe for images in the directory
import os, fnmatch

listOfFiles = os.listdir('../Data/S2_img_AC_acolite_2014-2019')
pattern = "*_L2W.nc"
img_list = []
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            img_list.append(entry)
        
img_dates = [i[8:18] for i in img_list]   
number_of_images = len(img_list)
sat_dates = pd.DataFrame(data = img_dates, columns = ['Date'], index = range(0, number_of_images))
sat_dates['#img'] = range(0, number_of_images)
sat_dates.head()


,Date,#img
0,2017_04_08,0
1,2017_04_15,1
2,2017_04_28,2
3,2017_05_05,3
4,2017_05_18,4


In [42]:
sat_dates['Date'] = pd.to_datetime(sat_dates['Date'], format='%Y_%m_%d')

In [43]:
# create a dataframe from merging chla data and image names (based on same date)
merged_data = pd.merge(chl_data, sat_dates, on = 'Date')
merged_data

,Date,SiteNumber,latitude,Longitude,Lab_Chla,#img
0,2017-05-18,1,50.586139,-105.384833,3.033477,4
1,2017-06-07,1,50.586139,-105.384833,3.033477,5
2,2017-07-04,1,50.586139,-105.384833,5.595528,6
3,2017-07-04,3,50.597500,-105.408000,7.290000,6
4,2017-07-17,1,50.586139,-105.384833,20.093948,7
...,...,...,...,...,...,...
102,2019-09-07,1,50.586139,-105.384833,55.694448,67
103,2019-09-15,1,50.586139,-105.384833,39.422214,38
104,2019-09-20,1,50.586139,-105.384833,36.803339,68
105,2019-09-22,1,50.586139,-105.384833,33.663162,39


In [44]:
# extracting reflectance values of correspond pixels to in-situ data
from netCDF4 import Dataset
Ref = [[]*0]*0 
for index, row in merged_data.iterrows():
    dataset = Dataset('../Data/S2_img_AC_acolite_2014-2019/' + img_list[merged_data.loc[index]['#img']])
    keys = dataset.variables.keys()
    dataset_1D = {}
    for key in keys:
        dataset_1D[key] = dataset.variables[key][:].flatten()

    dist = ((dataset_1D['lon'][:] - row['Longitude']) ** 2 + (dataset_1D['lat'][:] - row['latitude']) ** 2) ** (1/2) 
    idx_mins = dist.argsort()[:6]
    
    Rrs = []
    for key in keys:
        if 'Rrs' in key:        
            Rrs.append((dataset_1D[key][idx_mins]).mean())
    Ref.append(Rrs)

In [45]:
# name of columns to add to dataframe
added_ref = ['Rrs_443', 'Rrs_492', 'Rrs_560' , 'Rrs_665', 'Rrs_704', 'Rrs_740', 'Rrs_783', 'Rrs_833', 'Rrs_865', 'Rrs_1614' , 'Rrs_2202']

In [46]:
# crate a dataframe of ref data
sat_ref = pd.DataFrame(data = Ref, columns = added_ref)
sat_ref

,Rrs_443,Rrs_492,Rrs_560,Rrs_665,Rrs_704,Rrs_740,Rrs_783,Rrs_833,Rrs_865,Rrs_1614,Rrs_2202
0,-0.000210,0.002320,0.008061,0.002002,0.002088,0.001217,0.001300,0.001376,0.001050,0.000141,0.000044
1,-0.000399,0.002583,0.010152,0.002968,0.002678,0.001908,0.001909,0.001841,0.001653,0.000230,0.000001
2,0.000948,0.003612,0.010687,0.003895,0.003828,0.003567,0.003550,0.002877,0.002930,0.000400,0.000000
3,0.000852,0.003128,0.010139,0.003633,0.003538,0.003553,0.003564,0.002621,0.002903,0.000239,0.000000
4,0.014966,0.015721,0.019980,0.012158,0.014999,0.018366,0.017922,0.016363,0.015386,0.000510,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merge sat and field data


In [47]:
# concating chla and ref dataframes 
samples = pd.concat([merged_data, sat_ref], axis = 1)
print(samples.shape)
samples

(107, 17)


,Date,SiteNumber,latitude,Longitude,Lab_Chla,#img,Rrs_443,Rrs_492,Rrs_560,Rrs_665,Rrs_704,Rrs_740,Rrs_783,Rrs_833,Rrs_865,Rrs_1614,Rrs_2202
0,2017-05-18,1,50.586139,-105.384833,3.033477,4,-0.000210,0.002320,0.008061,0.002002,0.002088,0.001217,0.001300,0.001376,0.001050,0.000141,0.000044
1,2017-06-07,1,50.586139,-105.384833,3.033477,5,-0.000399,0.002583,0.010152,0.002968,0.002678,0.001908,0.001909,0.001841,0.001653,0.000230,0.000001
2,2017-07-04,1,50.586139,-105.384833,5.595528,6,0.000948,0.003612,0.010687,0.003895,0.003828,0.003567,0.003550,0.002877,0.002930,0.000400,0.000000
3,2017-07-04,3,50.597500,-105.408000,7.290000,6,0.000852,0.003128,0.010139,0.003633,0.003538,0.003553,0.003564,0.002621,0.002903,0.000239,0.000000
4,2017-07-17,1,50.586139,-105.384833,20.093948,7,0.014966,0.015721,0.019980,0.012158,0.014999,0.018366,0.017922,0.016363,0.015386,0.000510,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,2019-09-07,1,50.586139,-105.384833,55.694448,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,2019-09-15,1,50.586139,-105.384833,39.422214,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,2019-09-20,1,50.586139,-105.384833,36.803339,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,2019-09-22,1,50.586139,-105.384833,33.663162,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#drop missing values in ref 
samples.dropna(axis=0, inplace=True)
samples

,Date,SiteNumber,latitude,Longitude,Lab_Chla,#img,Rrs_443,Rrs_492,Rrs_560,Rrs_665,Rrs_704,Rrs_740,Rrs_783,Rrs_833,Rrs_865,Rrs_1614,Rrs_2202
0,2017-05-18,1,50.586139,-105.384833,3.033477,4,-0.000210,0.002320,0.008061,0.002002,0.002088,0.001217,0.001300,0.001376,0.001050,0.000141,0.000044
1,2017-06-07,1,50.586139,-105.384833,3.033477,5,-0.000399,0.002583,0.010152,0.002968,0.002678,0.001908,0.001909,0.001841,0.001653,0.000230,0.000001
2,2017-07-04,1,50.586139,-105.384833,5.595528,6,0.000948,0.003612,0.010687,0.003895,0.003828,0.003567,0.003550,0.002877,0.002930,0.000400,0.000000
3,2017-07-04,3,50.597500,-105.408000,7.290000,6,0.000852,0.003128,0.010139,0.003633,0.003538,0.003553,0.003564,0.002621,0.002903,0.000239,0.000000
4,2017-07-17,1,50.586139,-105.384833,20.093948,7,0.014966,0.015721,0.019980,0.012158,0.014999,0.018366,0.017922,0.016363,0.015386,0.000510,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019-07-29,1,50.586139,-105.384833,77.024402,63,0.003115,0.007206,0.018355,0.007344,0.017352,0.010898,0.011274,0.009329,0.007545,0.000096,0.000144
97,2019-08-11,1,50.586139,-105.384833,119.056578,64,0.004274,0.008520,0.020586,0.009508,0.020142,0.011754,0.011661,0.009617,0.007373,0.000105,0.000008
98,2019-08-18,1,50.586139,-105.384833,117.997073,65,0.001956,0.006759,0.018749,0.007857,0.018365,0.010210,0.010034,0.008219,0.006533,0.000057,0.000176
99,2019-08-21,1,50.586139,-105.384833,100.529651,66,0.003879,0.007416,0.020623,0.008424,0.018861,0.010246,0.010317,0.007981,0.006815,0.000014,0.000082


# Implement MDN

In [49]:
# prepare and save data to import to MDN
sat_ref_MDN = np.array(sat_ref.dropna(axis=0))[:,:7]
print(sat_ref_MDN)
np.savetxt("sat_ref_MDN.csv", sat_ref_MDN, delimiter=",")

[[-2.09765640e-04  2.31973897e-03  8.06056708e-03  2.00239499e-03
   2.08752463e-03  1.21714722e-03  1.30019512e-03]
 [-3.99375305e-04  2.58346763e-03  1.01522617e-02  2.96770618e-03
   2.67801806e-03  1.90786261e-03  1.90896029e-03]
 [ 9.48206696e-04  3.61236162e-03  1.06873764e-02  3.89533490e-03
   3.82831879e-03  3.56730237e-03  3.54983029e-03]
 [ 8.51780933e-04  3.12810019e-03  1.01393424e-02  3.63314780e-03
   3.53770959e-03  3.55285197e-03  3.56430630e-03]
 [ 1.49659170e-02  1.57210678e-02  1.99800078e-02  1.21579515e-02
   1.49987200e-02  1.83660220e-02  1.79224927e-02]
 [ 1.52818887e-02  1.60253663e-02  2.12811250e-02  1.33921690e-02
   1.68580022e-02  1.95817556e-02  1.89502686e-02]
 [ 1.06544383e-02  1.25398366e-02  1.69702452e-02  1.03156148e-02
   1.25483228e-02  1.72038171e-02  1.82624850e-02]
 [ 1.07370235e-03  3.94613715e-03  1.22233583e-02  5.90970926e-03
   7.70800002e-03  4.24789498e-03  3.84034286e-03]
 [ 7.11516710e-04  3.70116881e-03  1.20223342e-02  5.85199846e-0

In [50]:
# Run MDN 
! python -m MDN --sensor S2A sat_ref_MDN.csv

Min Rrs: [-0.00177937  0.00084586  0.00492017  0.00109415  0.00117368  0.00092856
  0.00099052]
Max Rrs: [0.01528189 0.01602537 0.02183341 0.01489365 0.02050217 0.01958176
 0.01895027]
Generating estimates for 85 data points
Min: [5.209888]
Max: [157.77429]
Saving estimates at location "MDN_sat_ref_MDN.csv"



100%|##########| 10/10 [00:35<00:00,  3.53s/it]


In [51]:
# read the output of MDN model and make format consistent with other models' outputs 
y_pred = pd.read_csv('MDN_sat_ref_MDN.csv')
y_pred = np.transpose(np.array(np.log10(y_pred)))
y_pred

array([[1.32628516, 1.248368  , 1.07146654, 1.13672469, 1.71897503,
        1.83735583, 1.74451895, 1.68740277, 1.66135705, 2.14498798,
        2.14030194, 1.95234145, 1.11372528, 0.71682839, 0.89824412,
        0.98115398, 0.7353698 , 1.23906635, 1.15433708, 1.9059268 ,
        2.16703072, 2.18350695, 2.19803623, 2.05738578, 1.6874759 ,
        1.72284978, 0.93889177, 1.91344025, 1.61222741, 1.25280851,
        1.17368897, 1.20888663, 1.19518439, 1.28083648, 1.33932414,
        1.4143271 , 1.15228623, 1.6252262 , 1.541398  , 1.53679623,
        1.38158262, 1.60601109, 2.04428899, 1.61088949, 1.79297526,
        1.59594979, 1.74128368, 1.71188655, 1.75027719, 1.7917345 ,
        1.63098457, 1.64834469, 1.46393903, 1.46433737, 1.06693098,
        1.18049051, 1.21849186, 1.15069011, 1.24556313, 1.2412626 ,
        1.30043354, 0.87604632, 1.4329483 , 1.19412375, 1.26983371,
        1.67695064, 1.69038734, 1.77712304, 1.89343548, 1.98199664,
        1.47343796, 1.26516182, 0.84286887, 1.34

# Model development

In [52]:
y = samples.Lab_Chla
X = samples.drop('Lab_Chla', axis = 1)
X.head()

,Date,SiteNumber,latitude,Longitude,#img,Rrs_443,Rrs_492,Rrs_560,Rrs_665,Rrs_704,Rrs_740,Rrs_783,Rrs_833,Rrs_865,Rrs_1614,Rrs_2202
0,2017-05-18,1,50.586139,-105.384833,4,-0.000210,0.002320,0.008061,0.002002,0.002088,0.001217,0.001300,0.001376,0.001050,0.000141,0.000044
1,2017-06-07,1,50.586139,-105.384833,5,-0.000399,0.002583,0.010152,0.002968,0.002678,0.001908,0.001909,0.001841,0.001653,0.000230,0.000001
2,2017-07-04,1,50.586139,-105.384833,6,0.000948,0.003612,0.010687,0.003895,0.003828,0.003567,0.003550,0.002877,0.002930,0.000400,0.000000
3,2017-07-04,3,50.597500,-105.408000,6,0.000852,0.003128,0.010139,0.003633,0.003538,0.003553,0.003564,0.002621,0.002903,0.000239,0.000000
4,2017-07-17,1,50.586139,-105.384833,7,0.014966,0.015721,0.019980,0.012158,0.014999,0.018366,0.017922,0.016363,0.015386,0.000510,0.000000


In [53]:
y_trans_log10 = np.log10(y)

In [54]:
# computing traditional models
X['NDCI'] = (X['Rrs_704']- X['Rrs_665']) / (X['Rrs_704']+  X['Rrs_665'])
X['M09'] = X['Rrs_704'] / X['Rrs_665']
#X['T07'] = X['Rrs_665'] / X['Rrs_560']
X['D05'] = (X['Rrs_665']**(-1) - X['Rrs_704']**(-1)) * X['Rrs_740']
X['OC2'] = X['Rrs_492'] / X['Rrs_560']
X['OC3'] = pd.concat([X['Rrs_492'],X['Rrs_443']], axis=1).max(axis=1) / X['Rrs_560']  

In [55]:
selected_features = []
#selected_features = added_ref[:5]
#selected_features.append('SiteNumber')
#selected_features.append('NDCI')
#selected_features.append('M09')
#selected_features.append('D05')
X_sel = pd.DataFrame(X[selected_features])
X_sel

""
0
1
2
3
4
...
95
97
98
99


In [56]:
# filt_outliers = X['Rrs_443'].between(X['Rrs_443'].quantile(.05), X['Rrs_443'].quantile(.95))
# X[filt_outliers].T.plot()
# X = X[filt_outliers]
# y = y[filt_outliers]

In [57]:
# from sklearn.preprocessing import scale 
# X_scaled = scale(X)
# X_scaled

In [58]:
## import models
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor
#from xgboost import XGBRegressor
#from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
#from sklearn.cross_decomposition import PLSRegression

#model = DecisionTreeRegressor(max_depth=100)
#model = RandomForestRegressor(n_estimators=50, random_state=0)
#model = XGBRegressor(booster='gblinear',n_estimators=500, random_state=0, learning_rate = 0.1)
#model = SVR(kernel = 'rbf')
model = LinearRegression()
#model = PLSRegression(n_components = 4)

In [59]:
# create a pipeline
from sklearn.pipeline import Pipeline
my_pipeline = Pipeline(steps=[('model', model)])

In [60]:
# cross validation and accuracy assessment of model
from sklearn.model_selection import cross_val_score
mae = 10**(-1 * cross_val_score(my_pipeline, X_sel, y_trans_log10, cv=5, scoring='neg_mean_absolute_error').mean())
mae

C:\Users\amchegoo\Anaconda3\envs\MDN\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Found array with 0 feature(s) (shape=(68, 0)) while a minimum of 1 is required.

# validation performance

In [ ]:
# getting predictions from cross validation
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(my_pipeline, X_sel, y_trans_log10, cv=5)

In [ ]:
from scipy.io import savemat
savemat('y_pred.mat', {'y_pred': y_pred})
#y_trans_log10[y_trans_log10 == 0] = 0.05
savemat('y.mat', {'y': np.array(y_trans_log10)})

In [ ]:
img_list[62]

In [ ]:
from netCDF4 import Dataset
#Ref = [[]*0]*0 
#for index, row in merged_data.iterrows():
dataset = Dataset('../S2_img_AC_acolite_2014-2019/' + img_list[62])
keys = dataset.variables.keys()
dataset_1D = {}
for key in keys:
    dataset_1D[key] = dataset.variables[key][:].flatten()
    


In [ ]:
Ref = [[]*0]*0 
for i in range(0,len(dataset_1D['lat'])):
    if dataset_1D['l2_flags'][i] == 0:
        Rrs = []
        for key in keys:
            if 'Rrs' in key:        
                Rrs.append(dataset_1D[key][i])
        Ref.append(Rrs)
        
ww = (np.array(Ref))   
ww = ww[:, :6]

In [ ]:
ww

In [ ]:
# from sklearn.preprocessing import scale 
# ww_scaled = scale(ww)
# ww_scaled

In [ ]:
def reject_outliers(data, m):
    return abs(data - np.median(data)) > m * np.std(data)

idx = ww.copy()
for i in range(0, ww.shape[1]):
    data = ww[:,i]
    idx[:,i] = reject_outliers(data, 2)

idx_outlier = idx.sum(axis=1) != 0

In [ ]:
sum(idx_outlier)

In [ ]:
ww

In [ ]:
model.fit(X, y_trans_log10)
values_pred = model.predict(ww)
idx_values_pred_outlier = reject_outliers(values_pred, 2)
values_pred[idx_values_pred_outlier] = 'NaN'
values_pred_unlog = 10 ** values_pred
values_pred_unlog[idx_outlier] = 'NaN'


In [ ]:
sum(idx_values_pred_outlier)

In [ ]:
image_flat = np.array(dataset_1D['Rrs_492']) 
idx = np.isnan(image_flat)
image_flat[~idx]= values_pred_unlog

In [ ]:
image = image_flat.reshape(dataset.dimensions['y'].size, dataset.dimensions['x'].size)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

ax = plt.figure(figsize = [16,12])
heatmap = plt.imshow(image, cmap='jet', interpolation='nearest')

cbar = plt.colorbar(heatmap, cax = ax.add_axes([.15, 0.15, 0.03, 0.7]))
cbar.ax.tick_params(labelsize=20) 

In [ ]:
ax.savefig('SVR_62.png',dpi=500, format='png')